In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from google.cloud import aiplatform
from langchain_google_vertexai import VertexAIEmbeddings
import PyPDF2
from google.generativeai import configure, GenerativeModel

import google.generativeai as genai
import pandas as pd
from google import genai


In [54]:

from IPython.display import Image, display
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict, Dict, List
from langchain_google_genai import ChatGoogleGenerativeAI
from logging import getLogger, INFO, Formatter, LoggerAdapter, StreamHandler
import sys
import asyncio
import json
import nest_asyncio
nest_asyncio.apply()


####Logger

In [38]:
# ---------- LOGGER Implementation ----------
def get_logger(name, level=INFO):
    handler = StreamHandler(sys.stdout)
    log_format = " %(levelname)s : --> %(message)s"
    formatter = Formatter(log_format)
    handler.setFormatter(formatter)
    logger = getLogger(name)
    logger.addHandler(handler)
    logger.setLevel(level)
    return logger

#### Data Models for Pydantic Validation of LLM Responses

In [39]:
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class ValidatorOutput(BaseModel):
    test_id: int
    test_name: str
    test_case_description: str
    
class FinalOutput(BaseModel):
    test_cases: list[ValidatorOutput]
    
output_parser = PydanticOutputParser(pydantic_object=FinalOutput)

In [40]:
# Sample JSON string
json_str = """
{
  "test_cases": [
    {
      "test_id": 1,
      "test_name": "Login Functionality",
      "test_case_description": "Verify that a user can log in with valid credentials."
    },
    {
      "test_id": 2,
      "test_name": "Password Reset",
      "test_case_description": "Ensure the password reset link works and updates the password."
    }
  ]
}
"""

In [41]:
output = output_parser.parse(json_str)

In [42]:
type(output)

__main__.FinalOutput

In [43]:
output.model_dump()

{'test_cases': [{'test_id': 1,
   'test_name': 'Login Functionality',
   'test_case_description': 'Verify that a user can log in with valid credentials.'},
  {'test_id': 2,
   'test_name': 'Password Reset',
   'test_case_description': 'Ensure the password reset link works and updates the password.'}]}

#### Models Initialization

In [63]:
## Models Initialization
llm_eval = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=1,api_key=os.getenv("GEMINI_API_KEY"))


response = llm_eval.invoke("Hello, can you hear me?")
print(response)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised PermissionDenied: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.GenerateContent"
}
].


PermissionDenied: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.GenerateContent"
}
]

In [45]:
#DOcument extract
document = 'E:\CODING\GenAI\TestCases.pdf'
def read_pdf(document):
    """Extract text from a PDF file"""
    text = ""
    with open(document, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def read_docx(file_path):
    """Extract text from a Word file"""
    doc = Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])

def read_txt(file_path):
    """Extract text from a TXT file"""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

In [46]:
read_pdf(document)

'Product Requirement Document (PRD)  \nProject Name: MedBillPro – Medical Billing Software  \nVersion: 1.0  \nPrepared For: QA Team  \nDate: [Insert Date]  \nPrepared By: [Your Name/Team]  \n1. Purpose  \nThe purpose of MedBillPro is to streamline medical billing, insurance claim management, \nand patient invoicing for healthcare providers. The QA team should validate functionality, \nworkflows, compliance with HIPAA/GDPR, accuracy of billing calculations, and system \nintegrations.  \n2. Scope  \n\uf0b7 Manage patient records and billing infor mation.  \n\uf0b7 Automate claim generation and submission to insurers.  \n\uf0b7 Support multiple payment methods (cash, card, UPI, net banking, insurance).  \n\uf0b7 Generate invoices, receipts, and audit logs.  \n\uf0b7 Ensure compliance with healthcare standards.  \n3. Functional Modules  \n3.1 Pa tient Management  \n\uf0b7 Add, edit, delete patient records.  \n\uf0b7 Unique Patient ID auto -generated.  \n3.2 Appointment & Service Entry  \n\

In [47]:
def qc_doc(document):
    """
    function to generate test case based on the given document .
     
    """
    prompt = f"""
            
            You are a **QA test designer**. Analyze the uploaded PRD (Product Requirement Document) and functionality documents. From these, generate **all possible high-level QA test cases**, covering:  

            - Functional scenarios  
            - Negative scenarios  
            - Edge cases  
            - Performance considerations  
            - Security validations  
            input document:{document}

           Output the test cases in the following structured template:  

            ---
            **Category:** [Functional | Negative | Edge | Performance | Security]  
            **Test Case ID:** [Unique ID]  
            **Title:** [Concise test case title]  
            **Description:** [High-level explanation of what to validate]  
            **Preconditions:** [If any]  
            **Expected Outcome:** [What should happen]  
            ---

            **Guidelines:**  
            - Do not go into step-by-step detail; keep them **high-level** for QA engineers to refine.  
            - Ensure **completeness** by covering all categories.  
            - Maintain clarity and consistency in wording.  
            - Where requirements are ambiguous, propose multiple possible test cases.  
                # Example QC results per chunk
                """
    llm_output = llm_eval.invoke(prompt).content
    print(f"LLM Quality Check (Test case generation)Output Completed")
    return llm_output
    #parse json string
   

In [49]:
qc_doc(document)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised PermissionDenied: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.GenerateContent"
}
].


PermissionDenied: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.GenerateContent"
}
]

####Validation model

In [ ]:
def validation(document, llm_output):
    """
    Validation .
     
    """
    prompt = f'''You are a **QA test case validator**. Your task is to validate the test cases generated from a Product Requirement Document (PRD) and functionality documents.  
            Input Document: {document}
            Generated Test case {llm_output}
            ### **Validation Scope:**  
            - Ensure test cases cover **all categories**: Functional, Negative, Edge, Performance, Security.  
            - Verify each test case includes the required fields:  
            - test_id  
            - test_case  
            - test_description  
            - expected_outcome  
            - test_severity (Low / High / Critical)  
            - Identify **gaps**: missing categories, vague wording, missing outcomes, or unclear severity.  
            - Detect **duplicates or overlaps**.  
            - Suggest **improvements** for incomplete or ambiguous cases.  

            ### **Output Format:**  
            Return the validation results **strictly as a list of dictionaries**, with each dictionary containing the following fields:  

            ```json
            [
            {
                "test_id": "TC001",
                "test_case": "Test Case Name",
                "test_description": "Test Case description.",
                "expected_outcome": "TEst case outcome",
                "test_severity": "low/Medium/High/Critical"
            }
            ]
            ```
            ### **Guidelines:**  

            * Include only validated test cases in the output.  
            *  If issues are found (e.g., missing field, unclear description), **correct them before adding** to the dictionary.  
            *  If a category is missing (Functional, Negative, Edge, Performance, Security), **add new test cases** to cover it.  
            * Maintain **clarity, consistency, and completeness**.  
            * Assign **test_severity** logically:  

            * **Critical** → Security, authentication, and data integrity cases.  
            * **High** → Core functional and performance cases.  
            * **Low** → Minor UI/UX or non-blocking cases.  
            '''
    llm_output = llm_eval.invoke(prompt).content
    print(f"TEst validator Output Completed")
    return llm_output